# 3rd Level Model Structure: Single Stage Reactive Distillation

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))

import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets

In [2]:
from pyomo import environ as pe
from modules.global_set import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule

from utility.display_utility import trans_product_mole, trans_product_mass, beautify2
from utility.model_utility import add_dual, update_dual, check_DOF
from utility.data_utility import cal_cnumber

model = pe.ConcreteModel()

# Global Set

In [3]:
model.TRAY = pe.RangeSet(1,3)

# Construct Reactive Stages

In [4]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].T_F
| reactive[1].P
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].T
| reactive[1].H_F
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].r_total_comp
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_block.r_WGS
| reactive[1].kinetics_bloc

# Construct a single condenser

In [5]:
model.condenser = pe.Block(rule=condenser_stage_rule)

| Importing Condenser Stage......
| Adding the following local variable:
------------------------------------
| condenser.T
| condenser.T_F
| condenser.P
| condenser.Q_main
| condenser.x_
| condenser.y_
| condenser.x
| condenser.y
| condenser.z
| condenser.L
| condenser.W
| condenser.V
| condenser.F
| condenser.H_L_
| condenser.H_V_
| condenser.H_L
| condenser.H_V
| condenser.H_F
| condenser.f_V
| condenser.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.energy_block.dH_F
| condenser.energy_block.dH_V
| condenser.energy_block.dH_L
| condenser.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.VLE_block.n_ave
| condenser.VLE_block.n_ave_cal
| condenser.VLE_block.Hen
| condenser.VLE_block.Hen0
| condenser.VLE_blo

# Linking Stage Variables

### Vapor Between Reactive Stages

In [6]:
def V_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].V['in'] == model.reactive[j+1].V['out']
model.V_between_con = pe.Constraint(model.TRAY,rule=V_between_rule)

def Vy_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].y_['in',i] == model.reactive[j+1].y[i]
model.Vy_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Vy_between_rule)

def Vh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].H_V_['in'] == model.reactive[j+1].H_V
model.Vh_between_con = pe.Constraint(model.TRAY,rule=Vh_between_rule)

### Liquid Between Reactive Stages

In [7]:
def L_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].L['in'] == model.reactive[j].L['out']
model.L_between_con = pe.Constraint(model.TRAY,rule=L_between_rule)

def Lx_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].x_['in',i] == model.reactive[j].x[i]
model.Ly_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Lx_between_rule)

def Lh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].H_L_['in'] == model.reactive[j].H_L
model.Lh_between_con = pe.Constraint(model.TRAY,rule=Lh_between_rule)

### Condenser

In [8]:
def V_condenser_rule(model):
    return model.reactive[model.TRAY.first()].V['out'] == model.condenser.V['in']
model.V_condenser_con = pe.Constraint(rule=V_condenser_rule)

def Vy_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].y[i] == model.condenser.y_['in',i]
model.Vy_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_condenser_rule)

def Vh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_V == model.condenser.H_V_['in']
model.Vh_condenser_con = pe.Constraint(rule=Vh_condenser_rule)

In [9]:
def L_condenser_rule(model):
    return model.reactive[model.TRAY.first()].L['in'] == model.condenser.L['out']
model.L_condenser_con = pe.Constraint(rule=L_condenser_rule)

def Lx_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].x_['in',i] == model.condenser.x[i]
model.Lx_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_condenser_rule)

def Lh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_L_['in'] == model.condenser.H_L
model.Lh_condenser_con = pe.Constraint(rule=Lh_condenser_rule)

In [10]:
model.obj = pe.Objective(expr = sum(model.reactive[j].T for j in model.TRAY) ,sense=pe.maximize)

# Load from single stage solutions

In [11]:
with open('../saved_solutions/1_stage_condenser_adiabatic.pickle', 'rb') as f:
    results_imported = pickle.load(f)
results_changed = results_imported

### Duplicate variable solution and bounds multiplier

In [12]:
for i in list(results_imported.Solution.Variable.keys()):
    if i.startswith('reactive[1].'):
        for j in model.TRAY:
            if j != 1:
                results_changed.Solution.Variable[i.replace('reactive[1].','reactive[{}].'.format(j))] = \
                results_imported.Solution.Variable[i]

### Duplicate constraint multiplier

In [13]:
for i in list(results_changed.Solution.Constraint.keys()):
    if i.startswith('reactive[1].'):
        for j in model.TRAY:
            if j != 1:
                results_changed.Solution.Constraint[i.replace('reactive[1].','reactive[{}].'.format(j))] = \
                results_imported.Solution.Constraint[i]

### Load changed solution into current model

In [14]:
model.solutions.load_from(results_changed)

# Fixing Redundent Stream Variables

In [15]:
# condenser
model.condenser.VLE_block.n_ave.fix(4)

model.condenser.F.fix(0)
model.condenser.T_F.fix(300+273.15)
model.condenser.z.fix(0)

model.condenser.V['P'].fix(0)
model.condenser.L['in'].fix(0)
for i in m.COMP_TOTAL: model.condenser.x_['in',i].fix(0)
model.condenser.H_L_['in'].fix(0)

In [16]:
# 'reboiler' fixing last stage V_in

model.reactive[model.TRAY.last()].V['in'].fix(0)
for i in m.COMP_TOTAL: model.reactive[model.TRAY.last()].y_['in',i].fix(0)
model.reactive[model.TRAY.last()].H_V_['in'].fix(0)

# Load Operating Parameters

In [17]:
# condenser
model.condenser.P.fix(19)
# model.condenser.T.fix(30+273.15)
model.condenser.T.fix(30+273.15)
model.condenser.L['out'].fix(0)

# reactive stage
for j in model.reactive:
    model.reactive[j].cat.fix(5000)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(20)
    
    model.reactive[j].F.fix(1)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(1/(1+2))
    model.reactive[j].z['H2'].fix(2/(1+2))
    model.reactive[j].z['C30H62'].fix(0)
    model.reactive[j].V['P'].fix(0)
    model.reactive[j].L['P'].fix(0)
    # model.reactive[j].Q_main.fix(0)
    model.reactive[j].T.setub(240+273.15)

In [18]:
check_DOF(pe,model)

Active Equality Constraints:	 4986
Active Inequality Constraints:	 0
Active Variables:		 5191
Fixed Variables:		 202
DOF:				 3


In [19]:
opt = pe.SolverFactory('ipopt')
opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'
# opt.options['nlp_scaling_method'] = None
# opt.options['constr_viol_tol'] = 1e-7
# opt.options['acceptable_constr_viol_tol'] = 1e-7
opt.options['max_iter'] = 7000
# opt.options['dual_inf_tol'] = '+inf'
# opt.options['acceptable_dual_inf_tol'] = '+inf'

In [20]:
add_dual(pe,model)
update_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


In [21]:
results = opt.solve(model,tee=True)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
max_iter=7000


List of user-set options:

                                    Name   Value                used
                           linear_solver = ma86                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver ma86.

Number of nonzeros in equality constraint Jacobian...:    16286
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.....

  77r-1.5394245e+03 2.13e+01 9.99e+02   0.5 0.00e+00    -  0.00e+00 4.92e-07R  4
  78r-1.5394245e+03 2.12e+01 9.98e+02   0.5 1.13e+03    -  1.13e-02 5.06e-06f  1
  79r-1.5394167e+03 2.33e+00 9.90e+02   0.5 9.06e+02    -  6.40e-03 8.21e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80 -1.5394214e+03 2.32e+00 1.91e+03  -1.0 1.83e+03    -  3.15e-04 8.69e-04f  1
  81 -1.5394215e+03 2.32e+00 4.34e+03  -1.0 2.23e+03    -  1.83e-03 1.21e-05h  1
  82 -1.5394228e+03 2.32e+00 4.49e+03  -1.0 2.12e+04    -  7.06e-05 2.61e-05h  1
  83 -1.5392044e+03 2.32e+00 2.53e+03  -1.0 5.79e+04    -  3.63e-05 6.06e-04h  1
  84 -1.5390777e+03 2.32e+00 6.81e+03  -1.0 7.15e+04    -  1.52e-03 2.72e-04h  1
  85 -1.5387723e+03 2.32e+00 1.11e+04  -1.0 7.77e+04    -  1.34e-03 5.98e-04h  1
  86r-1.5387723e+03 2.32e+00 9.99e+02   0.1 0.00e+00    -  0.00e+00 3.59e-07R  5
  87r-1.5387741e+03 2.31e+00 1.00e+03   0.1 5.41e+03    -  1.20e-02 5.23e-05f  1
  88r-1.5389615e+03 1.32e+00

In [22]:
update_dual(pe,model)

In [23]:
beautify2(pe,model)

Here comes the result:
----------------------------------------------------------------------------------------------------
		T		 Q			 V			 L
               30.00 	       -52.0845253731 	         0.3869491393 	         0.0000000000
              240.00 	       -46.3197171505 	         1.2891925483 	         0.0011185616
              240.00 	       -46.3197171505 	         0.8594616989 	         0.0022371232
              240.00 	       -46.3197171505 	         0.4297308494 	         0.0033556848
----------------------------------------------------------------------------------------------------
Top
V	 0.386949139265133
L	 0.09473439425247784
W	 0.807509014830218
----------------------------------------------------------------------------------------------------
Bottom
L	 0.0033556848383662057
----------------------------------------------------------------------------------------------------
Condenser:	Vapor		Liquid		Last Stage	Vapor		Liquid
H2 		50.198%		0.661%		 H2       	15.115%		0

In [ ]:
model.reactive[1].kinetics_block.r_FT_total.value

In [24]:
# model.solutions.store_to(results)
# with open('../saved_solutions/3_stage_condenser_240C.pickle','wb') as f:
#     pickle.dump(results,f)

In [25]:
model.condenser.L['out'].fixed = False
for j in model.reactive:
    model.reactive[j].T.fixed=True

In [31]:
model.del_component(model.obj)
model.obj = pe.Objective(expr = model.condenser.L['out'],sense=pe.maximize)

# So, what exactly does adding a reflux do?

In [26]:
Refluxrange = np.linspace(0,2,21)

In [30]:
cd_data = {};
cd_data['Re'] = []; cd_data['D'] = []; cd_data['V'] = []
cd_data['x'] = {}; cd_data['y'] = {}; cd_data['g'] = {}; cd_data['d'] = {};
for i in m.COMP_TOTAL:
    cd_data['x'][i] = []
    cd_data['y'][i] = []
    cd_data['g'][i] = []
    cd_data['d'][i] = []
    
rf_data = {}
for j in model.reactive:
    rf_data[j] = {}
    rf_data[j]['r'] = {}; rf_data[j]['b'] = {}; rf_data[j]['x'] = {};rf_data[j]['y'] = {};
    rf_data[j]['T'] = []; rf_data[j]['Q'] = []; rf_data[j]['V'] = []; rf_data[j]['L'] = []; 
    rf_data[j]['r_WGS'] = []; rf_data[j]['r_FT'] = []
    for i in m.COMP_TOTAL:
        rf_data[j]['r'][i] = []
        rf_data[j]['b'][i] = []
        rf_data[j]['x'][i] = []
        rf_data[j]['y'][i] = []       

In [33]:
for re in Refluxrange:
    # model.condenser.L['out'].fixed = False
    model.condenser.L['out'].setub(re)
    # model.reactive[1].T.fix(t)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    print('Solved\t|Reflux = {:.3f} kmol/s\t|Vapor = {:.3f} kmol/s\t|Distillate = {:.3f} kmol/s\t|Bottom = {:.3f} kmol/s'.\
          format(model.condenser.L['out'].value,model.condenser.V['out'].value,model.condenser.L['P'].value,model.reactive[3].L['out'].value))

    cd_data['V'].append(model.condenser.V['out'].value)
    cd_data['D'].append(model.condenser.L['P'].value)
    cd_data['Re'].append(model.condenser.L['out'].value)

    for i in model.reactive[1].r_total_comp:
        cd_data['x'][i].append(model.condenser.x[i].value)
        cd_data['y'][i].append(model.condenser.y[i].value)
        cd_data['g'][i].append(model.condenser.y[i].value*model.condenser.V['out'].value)
        cd_data['d'][i].append(model.condenser.x[i].value*model.condenser.L['P'].value)
    
    for j in model.reactive:      
        rf_data[j]['T'].append(model.reactive[j].T.value)
        rf_data[j]['Q'].append(model.reactive[j].Q_main.value)
        rf_data[j]['V'].append(model.reactive[j].V['out'].value)
        rf_data[j]['L'].append(model.reactive[j].L['out'].value)
        rf_data[j]['r_WGS'].append(model.reactive[j].kinetics_block.r_WGS.value)
        rf_data[j]['r_FT'].append(model.reactive[j].kinetics_block.r_FT_total.value)

        for i in model.reactive[1].r_total_comp:
            rf_data[j]['r'][i].append(model.reactive[j].r_total_comp[i].value)
            rf_data[j]['b'][i].append(model.reactive[j].x[i].value*model.reactive[j].L['out'].value)
            rf_data[j]['x'][i].append(model.reactive[j].x[i].value)
            rf_data[j]['y'][i].append(model.reactive[j].y[i].value)

Solved	|Reflux = 0.000 kmol/s	|Vapor = 0.387 kmol/s	|Distillate = 0.095 kmol/s	|Bottom = 0.003 kmol/s
Solved	|Reflux = 0.100 kmol/s	|Vapor = 0.393 kmol/s	|Distillate = 0.091 kmol/s	|Bottom = 0.006 kmol/s
Solved	|Reflux = 0.200 kmol/s	|Vapor = 0.399 kmol/s	|Distillate = 0.088 kmol/s	|Bottom = 0.009 kmol/s
Solved	|Reflux = 0.300 kmol/s	|Vapor = 0.405 kmol/s	|Distillate = 0.085 kmol/s	|Bottom = 0.011 kmol/s
Solved	|Reflux = 0.400 kmol/s	|Vapor = 0.411 kmol/s	|Distillate = 0.082 kmol/s	|Bottom = 0.013 kmol/s
Solved	|Reflux = 0.500 kmol/s	|Vapor = 0.417 kmol/s	|Distillate = 0.079 kmol/s	|Bottom = 0.015 kmol/s
Solved	|Reflux = 0.600 kmol/s	|Vapor = 0.422 kmol/s	|Distillate = 0.077 kmol/s	|Bottom = 0.017 kmol/s
Solved	|Reflux = 0.700 kmol/s	|Vapor = 0.428 kmol/s	|Distillate = 0.075 kmol/s	|Bottom = 0.018 kmol/s
Solved	|Reflux = 0.800 kmol/s	|Vapor = 0.433 kmol/s	|Distillate = 0.073 kmol/s	|Bottom = 0.019 kmol/s
Solved	|Reflux = 0.900 kmol/s	|Vapor = 0.438 kmol/s	|Distillate = 0.071 kmol/s	|Bo

In [47]:
cnumber_range = range(1,57)

In [39]:
def trans_cnumber(dic):
    molefraction = {}
    for i in range(1,57):
        molefraction[i] = []
    for i in m.COMP_ORG:
        molefraction[cal_cnumber(i)].append(np.array(dic[i]))
    for i in range(1,57):
        molefraction[i] = np.sum(molefraction[i],0)
    length = len(molefraction[1])
    tmp = {}
    for j in range(length):
        tmp[j] = []
        for i in range(1,57):
            tmp[j].append(molefraction[i][j])
    return tmp

In [60]:
g_data = trans_cnumber(cd_data['g'])
d_data = trans_cnumber(cd_data['d'])
b_data = trans_cnumber(rf_data[3]['b'])
# r_data = trans_cnumber(rf_data_master[t]['r'])

cd_x_data = trans_cnumber(cd_data['x'])
rf_x_data = {}
for j in model.reactive:
    rf_x_data[j] = trans_cnumber(rf_data[j]['x'])

In [76]:
 def plot_distribution(index):
    fig, (ax,ax2) = plt.subplots(2,1,figsize=(16,12))
    ax.plot(cnumber_range,g_data[index],'co-')
    ax.plot(cnumber_range,d_data[index],'go-')
    ax.plot(cnumber_range,b_data[index],'ro-')
    ax.set_yscale("log")
    ax.set_ylim(1e-12, 1)
    ax.legend(['Vapor','Distillate','Bottom'],fontsize=18)
    ax.set_title('T, Reflux {:.2f} kmol/s'.format(cd_data['Re'][index]),fontsize=18)

    ax.set_ylabel('Molar Flow (kmol/s)', color='K',fontsize=18)
    ax.set_xlabel('Carbon Number', color='K',fontsize=18)
    # ax.tick_params('y', colors='k',labelsize=18)
    # ax.tick_params('x', colors='k',labelsize=18)

    ax2.plot(cnumber_range,cd_x_data[index],'go-')
    ax2.plot(cnumber_range,rf_x_data[1][index],'co-')
    ax2.plot(cnumber_range,rf_x_data[2][index],'bo-')
    ax2.plot(cnumber_range,rf_x_data[3][index],'ro-')

    ax2.set_ylim(0, 0.2)
    ax2.legend(['Condenser','Stage 1','Stage 2','Stage 3'],fontsize=18)
    ax2.set_title('Liquid Composition (Mole)',fontsize=18)

    ax2.set_ylabel('Molar Fraction', color='K',fontsize=18)
    ax2.set_xlabel('Carbon Number', color='K',fontsize=18)

    ax.grid()
    ax2.grid()
    plt.show()

In [77]:
widgets.interact(plot_distribution, index = widgets.IntSlider(
    value=0,
    min=0,
    max=20,
    step=1,
    description='Reflux:',)
);

interactive(children=(IntSlider(value=0, description='Reflux:', max=20), Output()), _dom_classes=('widget-inte…